In [1]:
from fastai.vision import *
import fastai
from torchvision import transforms
import pandas as pd
from tqdm import tnrange, tqdm_notebook

In [3]:
STREETSTYLE_DIR = "/home/zerowitone/Desktop/Fellowship.ai"
main_dir = "/home/zerowitone/Desktop/Fellowship.ai/paperdoll"
down_dir = os.path.join(main_dir,'latent_features')
MANIFEST_FN = "streetstyle27k.manifest"
model_name = 'model'
model_version = 'V4'
labels = pd.read_csv(os.path.join(STREETSTYLE_DIR, MANIFEST_FN))
print("Found {} entries in {}".format(len(labels), MANIFEST_FN))
categories = ['clothing_pattern', 'major_color', 'wearing_necktie', 'collar_presence', 
              'wearing_scarf', 'sleeve_length', 'neckline_shape', 'clothing_category', 
              'wearing_jacket', 'wearing_hat', 'wearing_glasses', 'multiple_layers']
np.random.seed(1254)
torch.manual_seed(1254)



basedir = Path(STREETSTYLE_DIR)
csvpath = os.path.join(STREETSTYLE_DIR,'streetstyle.preprocess.csv')
print(csvpath)

ds_tfms = get_transforms()
input_size = (300, 200)
df = pd.read_csv(csvpath)
print(df.columns)
df_category = df[["images", categories[5]]]
street_data = (ImageList.from_df(df_category, path=basedir)
                     .split_by_rand_pct(0.2)
                     .label_from_df()
                     .transform(ds_tfms, size=input_size)
                     .databunch(bs=16)
                     .normalize(imagenet_stats))
print('{}'.format(categories[5]))

Found 27087 entries in streetstyle27k.manifest
/home/zerowitone/Desktop/Fellowship.ai/streetstyle.preprocess.csv
Index(['id', 'url', 'created_time', 'city_id', 'month_id', 'lat', 'long', 'x1',
       'y1', 'x2', 'y2', 'width', 'height', 'clothing_pattern', 'major_color',
       'wearing_necktie', 'collar_presence', 'wearing_scarf', 'sleeve_length',
       'neckline_shape', 'clothing_category', 'wearing_jacket', 'wearing_hat',
       'wearing_glasses', 'multiple_layers', 'images', 'fastai'],
      dtype='object')
sleeve_length


In [10]:
learn = load_learner(main_dir,main_dir+"/Unet_{}_{}".format(model_name, model_version))


In [38]:
def imsave(tensor, imsize=512, title=None):
    image = tensor.clone().cpu()
    image = image.view(*tensor.size())
    image = transforms.ToPILImage()(image)
    image.show()
    return image
    #print(np.asar

In [112]:
examples_dir =  os.path.join(STREETSTYLE_DIR,'Examples')
for i in range(10):
    index = np.random.randint(len(street_data.x), size=1)
    index = index[0]
    #print(index)

    _,__,pred=learn.predict(street_data.x[index])
    street_data.x[index].save(examples_dir+'/'+str(i)+'.png')
    imsave(pred[0:2]).save(examples_dir+'/mask_'+str(i)+'.png')

In [111]:

df_dict = defaultdict(list)
#print(df_category.tail)
counter = 0

print(len(street_data.x))
for i in tnrange(len(street_data.x), desc='predicting'):
    _,__,pred=learn.predict(street_data.x[i])
    img_msk = imsave(pred[0:2])
    path = os.path.join(down_dir,"{:07d}.png".format(counter))
    label = street_data.y[i]
    img_msk.save(path)
    df_dict['name'].append(str(path))
    df_dict['label'].append(str(label))
    counter+=1
df = pd.DataFrame.from_dict(df_dict)

21670


In [123]:
df.to_csv('labels.csv',index=False)

In [128]:
path = Path('./')
df = pd.read_csv( path/"labels.csv" )
df['name'] = df['name'].apply( lambda x: x.split("/")[-1] )

In [129]:
df.drop(df[df['label']== 'no_label'].index, inplace=True)
print(df.tail())
data = (ImageDataBunch.from_df(df=df, path=main_dir, folder='latent_features')
                     .split_by_rand_pct(0.2)
                     .label_from_df()
                     #.transform(ds_tfms, size=input_size)
                     .databunch(bs=4)
                     .normalize(imagenet_stats))

              name         label
20178  0021665.png  short_sleeve
20179  0021666.png   long_sleeve
20180  0021667.png   long_sleeve
20181  0021668.png  short_sleeve
20182  0021669.png  short_sleeve


In [130]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy, callback_fns=[callbacks.CSVLogger])

In [132]:
learn.lr_find() # find learning rate
learn.recorder.plot() # plot learning rate grap

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [133]:
learn.fit_one_cycle(6,  max_lr=slice(1e-06)) # train model

epoch,train_loss,valid_loss,accuracy,time
0,1.367012,1.209516,0.378136,03:32
1,1.455959,1.188962,0.385878,03:30
2,1.434841,1.210500,0.364509,03:30
3,1.410967,1.187335,0.386497,03:34
4,1.403802,1.183895,0.391762,03:39
5,1.425726,1.184104,0.383400,03:43


In [134]:
learn.unfreeze()
learn.recorder.plot()